# Demand Forecasting using Python

##### Import required Libaries


In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium

##### Load 1 week raw csv file into dataframe
##### dataset URL -> https://cycling.data.tfl.gov.uk/

In [2]:
raw_df = pd.read_csv("./data/195JourneyDataExtract01Jan2020-07Jan2020.csv").drop(columns=["Rental Id","Duration","Bike Id"])
raw_df.columns = raw_df.columns.str.replace(' ','_')

In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126801 entries, 0 to 126800
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   End_Date           126801 non-null  object
 1   EndStation_Id      126801 non-null  int64 
 2   EndStation_Name    126801 non-null  object
 3   Start_Date         126801 non-null  object
 4   StartStation_Id    126801 non-null  int64 
 5   StartStation_Name  126801 non-null  object
dtypes: int64(2), object(4)
memory usage: 5.8+ MB


In [4]:
raw_df.head()

,End_Date,EndStation_Id,EndStation_Name,Start_Date,StartStation_Id,StartStation_Name
0,07/01/2020 14:07,541,"Green Park Station, Mayfair",07/01/2020 13:51,164,"Cleveland Gardens, Bayswater"
1,07/01/2020 17:06,48,"Godliman Street, St. Paul's",07/01/2020 16:56,323,"Clifton Street, Shoreditch"
2,07/01/2020 11:01,654,"Ashmole Estate, Oval",07/01/2020 10:52,624,"Courland Grove, Wandsworth Road"
3,04/01/2020 12:27,685,"Osiers Road, Wandsworth",04/01/2020 12:17,774,"Hurlingham Park, Parsons Green"
4,02/01/2020 16:31,676,"Hartington Road, Stockwell",02/01/2020 16:12,83,"Panton Street, West End"


##### Get unique Station list from the entire dataframe.

##### Get Unique End Stations

In [5]:
uniq_end_station = raw_df.EndStation_Name.unique().tolist()

In [6]:
uniq_end_station

['Green Park Station, Mayfair',
 "Godliman Street, St. Paul's",
 'Ashmole Estate, Oval',
 'Osiers Road, Wandsworth',
 'Hartington Road, Stockwell',
 'Leman Street, Aldgate',
 'Ranelagh Gardens, Fulham',
 'Putney Rail Station, Putney',
 'Courland Grove, Wandsworth Road',
 'Clifton Street, Shoreditch',
 'Sandilands Road, Walham Green',
 'Panton Street, West End',
 'Smugglers Way, Wandsworth',
 'Kings Gate House, Westminster',
 'Argyll Road, Kensington',
 'Cranmer Road, Stockwell',
 'Buckingham Gate, Westminster',
 'Howick Place, Westminster',
 'Wellington Street , Strand',
 'Albert Square, Stockwell',
 'Strata, Elephant & Castle',
 'Doddington Grove, Kennington',
 'Bricklayers Arms, Borough',
 'New Kent Road, The Borough',
 'Great Suffolk Street, The Borough',
 'Garnet Street, Shadwell',
 'Braham Street, Aldgate',
 'Twig Folly Bridge, Mile End',
 'Westferry DLR, Limehouse',
 "Blackfriars Station, St. Paul's",
 'Reardon Street, Wapping',
 'Watney Street, Shadwell',
 'Tanner Street, Bermon

In [7]:
uniq_end_station_df = pd.DataFrame(uniq_end_station, columns=['Station'])

In [8]:
uniq_end_station_df.head()

,Station
0,"Green Park Station, Mayfair"
1,"Godliman Street, St. Paul's"
2,"Ashmole Estate, Oval"
3,"Osiers Road, Wandsworth"
4,"Hartington Road, Stockwell"


##### Get Unique Start Stations

In [9]:
uniq_start_station = raw_df.StartStation_Name.unique().tolist()

In [10]:
uniq_start_station

['Cleveland Gardens, Bayswater',
 'Clifton Street, Shoreditch',
 'Courland Grove, Wandsworth Road',
 'Hurlingham Park, Parsons Green',
 'Panton Street, West End',
 'Shadwell Station, Shadwell',
 'Osiers Road, Wandsworth',
 'Ashmole Estate, Oval',
 "Godliman Street, St. Paul's",
 'Smugglers Way, Wandsworth',
 'Teversham Lane, Stockwell',
 'Upper Richmond Road, Putney',
 'Sandilands Road, Walham Green',
 'New Kent Road, The Borough',
 'Harcourt Terrace, West Brompton',
 'Bankside Mix, Bankside',
 'Lower Marsh, Waterloo',
 'Southampton Street, Strand',
 'Howick Place, Westminster',
 'Cranmer Road, Stockwell',
 'Buckingham Gate, Westminster',
 'Kings Gate House, Westminster',
 'Walworth Road, Elephant & Castle',
 'Nesham Street, Wapping',
 'East Road, Hoxton',
 'Murray Grove , Hoxton',
 'Gower Place , Euston',
 'Webber Street , Southwark',
 'Chancery Lane, Holborn',
 'Gloucester Road (North), Kensington',
 'Old Quebec Street, Marylebone',
 'Claremont Square, Angel',
 'Great Percy Street, C

In [11]:
uniq_start_station_df = pd.DataFrame(uniq_start_station, columns=['Station'])

In [12]:
uniq_start_station_df.head()

,Station
0,"Cleveland Gardens, Bayswater"
1,"Clifton Street, Shoreditch"
2,"Courland Grove, Wandsworth Road"
3,"Hurlingham Park, Parsons Green"
4,"Panton Street, West End"


##### 1. Combine unique start and end station.
##### 2. Remove duplicates
##### 3. Sort alphabatically 
##### 4. Re-index

In [13]:
staging_stations = pd.concat([uniq_start_station_df, uniq_end_station_df], ignore_index=True, sort=True)

In [14]:
all_unique_stations = staging_stations.Station.unique().tolist()

In [15]:
all_unique_stations_df = pd.DataFrame(all_unique_stations, columns=['Station'])
all_unique_stations_df.sort_values(by=['Station'], inplace=True, ascending=True)
all_unique_stations_df.reset_index(drop=True, inplace=True)
all_unique_stations_df.head()

,Station
0,"Abbey Orchard Street, Westminster"
1,"Abbotsbury Road, Holland Park"
2,"Aberdeen Place, St. John's Wood"
3,"Aberfeldy Street, Poplar"
4,"Abingdon Green, Westminster"


##### Nominatim - Get Lat and Lon based on the address

In [16]:
geolocator = Nominatim(user_agent="tlf-app")

##### Sample Run test

In [17]:
geolocator.geocode("All Saints' Road, Portobello")

Location(All Saints' Road, All Saints Road, North Kensington, Royal Borough of Kensington and Chelsea, London, Kensington, Greater London, England, W11 1HF, United Kingdom, (51.5190755, -0.2046896, 0.0))

##### Iterate throught the unique station datafram to lookup lat and lon using Nominatim API service and save as CSV file.

In [18]:
import csv
import time

with open("stations_latlon.csv", "w", newline='') as file:
    writer = csv.writer(file, delimiter = "|")
    
    for index, row in all_unique_stations_df.iterrows():
        try:
            location = (geolocator.geocode(row["Station"]))
            writer.writerow((row["Station"],location.latitude,location.longitude))
        except:
            continue

##### Load Station Lat Lon csv file into dataframe

In [19]:
sat_lat_lon = pd.read_csv("stations_latlon.csv", sep = '|',header=None, names = ['station','lat','lon'])

In [20]:
sat_lat_lon.head()

,station,lat,lon
0,"Abbey Orchard Street, Westminster",51.498309,-0.131488
1,"Abbotsbury Road, Holland Park",51.501371,-0.205932
2,"Aberdeen Place, St. John's Wood",51.525419,-0.175263
3,"Aberfeldy Street, Poplar",51.513550,-0.005652
4,"Abingdon Green, Westminster",51.497615,-0.125966


##### Inner join raw dataframe with Station Lat Lon to assign Lat & Lon to the dataset

In [21]:
end_station_lat_lon = pd.merge(raw_df,sat_lat_lon,left_on='EndStation_Name', right_on='station',how='inner')

In [22]:
end_station_lat_lon.head()

,End_Date,EndStation_Id,EndStation_Name,Start_Date,StartStation_Id,StartStation_Name,station,lat,lon
0,07/01/2020 14:07,541,"Green Park Station, Mayfair",07/01/2020 13:51,164,"Cleveland Gardens, Bayswater","Green Park Station, Mayfair",51.506493,-0.142975
1,01/01/2020 14:36,541,"Green Park Station, Mayfair",01/01/2020 14:11,97,"Gloucester Road (North), Kensington","Green Park Station, Mayfair",51.506493,-0.142975
2,01/01/2020 02:03,541,"Green Park Station, Mayfair",01/01/2020 01:56,116,"Little Argyll Street, West End","Green Park Station, Mayfair",51.506493,-0.142975
3,01/01/2020 02:03,541,"Green Park Station, Mayfair",01/01/2020 01:56,116,"Little Argyll Street, West End","Green Park Station, Mayfair",51.506493,-0.142975
4,07/01/2020 06:31,541,"Green Park Station, Mayfair",07/01/2020 06:17,105,"Westbourne Grove, Bayswater","Green Park Station, Mayfair",51.506493,-0.142975


In [23]:
end_station_lat_lon.rename(columns={"lat":"end_lat","lon":"end_lon"},inplace=True)
end_station_lat_lon = end_station_lat_lon.drop('station', axis=1)
end_station_lat_lon.head()

,End_Date,EndStation_Id,EndStation_Name,Start_Date,StartStation_Id,StartStation_Name,end_lat,end_lon
0,07/01/2020 14:07,541,"Green Park Station, Mayfair",07/01/2020 13:51,164,"Cleveland Gardens, Bayswater",51.506493,-0.142975
1,01/01/2020 14:36,541,"Green Park Station, Mayfair",01/01/2020 14:11,97,"Gloucester Road (North), Kensington",51.506493,-0.142975
2,01/01/2020 02:03,541,"Green Park Station, Mayfair",01/01/2020 01:56,116,"Little Argyll Street, West End",51.506493,-0.142975
3,01/01/2020 02:03,541,"Green Park Station, Mayfair",01/01/2020 01:56,116,"Little Argyll Street, West End",51.506493,-0.142975
4,07/01/2020 06:31,541,"Green Park Station, Mayfair",07/01/2020 06:17,105,"Westbourne Grove, Bayswater",51.506493,-0.142975


In [24]:
final_station_lat_lon = pd.merge(end_station_lat_lon,sat_lat_lon,left_on='StartStation_Name', right_on='station',how='inner')

In [25]:
final_station_lat_lon = final_station_lat_lon.drop('station', axis=1)
final_station_lat_lon.rename(columns={"lat":"start_lat","lon":"start_lon"},inplace=True)
final_station_lat_lon.head()

,End_Date,EndStation_Id,EndStation_Name,Start_Date,StartStation_Id,StartStation_Name,end_lat,end_lon,start_lat,start_lon
0,07/01/2020 14:07,541,"Green Park Station, Mayfair",07/01/2020 13:51,164,"Cleveland Gardens, Bayswater",51.506493,-0.142975,51.515396,-0.1824
1,02/01/2020 06:54,528,"Clarges Street, Mayfair",02/01/2020 06:43,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824
2,03/01/2020 06:51,528,"Clarges Street, Mayfair",03/01/2020 06:39,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824
3,02/01/2020 06:52,528,"Clarges Street, Mayfair",02/01/2020 06:39,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824
4,06/01/2020 06:51,528,"Clarges Street, Mayfair",06/01/2020 06:38,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824


In [26]:
final_station_lat_lon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116308 entries, 0 to 116307
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   End_Date           116308 non-null  object 
 1   EndStation_Id      116308 non-null  int64  
 2   EndStation_Name    116308 non-null  object 
 3   Start_Date         116308 non-null  object 
 4   StartStation_Id    116308 non-null  int64  
 5   StartStation_Name  116308 non-null  object 
 6   end_lat            116308 non-null  float64
 7   end_lon            116308 non-null  float64
 8   start_lat          116308 non-null  float64
 9   start_lon          116308 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 9.8+ MB


##### Get Hour, Day, Week and Month from the dataset

In [27]:
final_station_lat_lon.Start_Date = pd.to_datetime(final_station_lat_lon.Start_Date, format='%d/%m/%Y %H:%M')
final_station_lat_lon['start_month'] = final_station_lat_lon.Start_Date.apply(lambda x: x.month)
final_station_lat_lon['start_week'] = final_station_lat_lon.Start_Date.apply(lambda x: x.week)
final_station_lat_lon['start_day'] = final_station_lat_lon.Start_Date.apply(lambda x: x.day)
final_station_lat_lon['start_hour'] = final_station_lat_lon.Start_Date.apply(lambda x: x.hour)

In [28]:
final_station_lat_lon.head()

,End_Date,EndStation_Id,EndStation_Name,Start_Date,StartStation_Id,StartStation_Name,end_lat,end_lon,start_lat,start_lon,start_month,start_week,start_day,start_hour
0,07/01/2020 14:07,541,"Green Park Station, Mayfair",2020-01-07 13:51:00,164,"Cleveland Gardens, Bayswater",51.506493,-0.142975,51.515396,-0.1824,1,2,7,13
1,02/01/2020 06:54,528,"Clarges Street, Mayfair",2020-01-02 06:43:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,1,2,6
2,03/01/2020 06:51,528,"Clarges Street, Mayfair",2020-01-03 06:39:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,1,3,6
3,02/01/2020 06:52,528,"Clarges Street, Mayfair",2020-01-02 06:39:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,1,2,6
4,06/01/2020 06:51,528,"Clarges Street, Mayfair",2020-01-06 06:38:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,2,6,6


##### Create a basemap with London as center

In [29]:
def generateBaseMap(default_location=[51.51222953559879, -0.09272574193488463], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map=generateBaseMap()
base_map

##### Add count column for aggreation. 

In [30]:
final_station_lat_lon['count'] = 1
final_station_lat_lon.head()

,End_Date,EndStation_Id,EndStation_Name,Start_Date,StartStation_Id,StartStation_Name,end_lat,end_lon,start_lat,start_lon,start_month,start_week,start_day,start_hour,count
0,07/01/2020 14:07,541,"Green Park Station, Mayfair",2020-01-07 13:51:00,164,"Cleveland Gardens, Bayswater",51.506493,-0.142975,51.515396,-0.1824,1,2,7,13,1
1,02/01/2020 06:54,528,"Clarges Street, Mayfair",2020-01-02 06:43:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,1,2,6,1
2,03/01/2020 06:51,528,"Clarges Street, Mayfair",2020-01-03 06:39:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,1,3,6,1
3,02/01/2020 06:52,528,"Clarges Street, Mayfair",2020-01-02 06:39:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,1,2,6,1
4,06/01/2020 06:51,528,"Clarges Street, Mayfair",2020-01-06 06:38:00,164,"Cleveland Gardens, Bayswater",51.507687,-0.146346,51.515396,-0.1824,1,2,6,6,1


##### Get the count of cycles starting point using group by and count.

In [31]:
final_station_lat_lon[['start_lat', 'start_lon', 'count']].groupby(['start_lat', 'start_lon']).sum()

,,count
start_lat,start_lon,
-45.702879,168.091561,929
-37.883781,145.067649,230
-37.789456,144.928671,85
-19.277057,146.796489,284
33.705898,-117.800375,377
...,...,...
52.008907,11.700334,129
52.111338,-1.920788,39
53.475009,-3.026259,537


##### Assign the result to a list.

In [32]:
final_station_lat_lon[['start_lat', 'start_lon', 'count']].groupby(['start_lat', 'start_lon']).sum().reset_index().values.tolist()

[[-45.7028786, 168.0915608, 929.0],
 [-37.883781, 145.0676495, 230.0],
 [-37.789456400000006, 144.92867057895717, 85.0],
 [-19.2770565, 146.7964885, 284.0],
 [33.7058983, -117.8003754, 377.0],
 [37.358528, -79.1780077, 94.0],
 [42.5118393, -92.4015733, 145.0],
 [43.46210175, -80.525300878738, 697.0],
 [48.3368254, -116.3950265, 145.0],
 [50.8725323, -2.964588663166145, 261.0],
 [51.4525657, -2.5727229, 202.0],
 [51.4544389, -0.1949463, 76.0],
 [51.4568295, -0.202782, 41.0],
 [51.457921, -0.1747085, 43.0],
 [51.4581682, -0.1920895, 179.0],
 [51.4587164, -0.1806832, 46.0],
 [51.4587796, -0.2010946, 28.0],
 [51.4593722, -0.1861135, 62.0],
 [51.4601828, -0.1902746, 32.0],
 [51.4602027, -0.1673095, 79.0],
 [51.4603182, -0.1146357, 113.0],
 [51.4607051, -0.2163943, 62.0],
 [51.4608587, -0.2124829, 35.0],
 [51.4609073, -0.1872281, 143.0],
 [51.4609339, -0.2153478, 48.0],
 [51.4613418, -0.1592604, 129.0],
 [51.4614275, -0.2006413, 72.0],
 [51.4616082, -0.1751516, 67.0],
 [51.4616316, -0.216565

##### Demo the distribution of the dataset.

In [33]:
from folium.plugins import HeatMap
base_map = generateBaseMap()
HeatMap(data=final_station_lat_lon[['start_lat', 'start_lon', 'count']].groupby(['start_lat', 'start_lon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map) 
base_map

##### Group the geo location based on count.

In [34]:
df_hour_list = []
for hour in final_station_lat_lon.start_hour.sort_values().unique():
    df_hour_list.append(final_station_lat_lon.loc[final_station_lat_lon.start_hour == hour, ['start_lat', 'start_lon', 'count']].groupby(['start_lat', 'start_lon']).sum().reset_index().values.tolist())


In [35]:
final_station_lat_lon.start_hour.sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int64)

In [36]:
df_hour_list

[[[-45.7028786, 168.0915608, 2.0],
  [-37.883781, 145.0676495, 2.0],
  [-19.2770565, 146.7964885, 1.0],
  [42.5118393, -92.4015733, 3.0],
  [50.8725323, -2.964588663166145, 1.0],
  [51.4525657, -2.5727229, 4.0],
  [51.4581682, -0.1920895, 1.0],
  [51.4587164, -0.1806832, 1.0],
  [51.4593722, -0.1861135, 1.0],
  [51.4607051, -0.2163943, 1.0],
  [51.4609339, -0.2153478, 2.0],
  [51.4613418, -0.1592604, 1.0],
  [51.4616316, -0.2165651, 2.0],
  [51.461687, -0.197573, 1.0],
  [51.4618939, -0.1675653, 3.0],
  [51.462434, -0.1893883, 1.0],
  [51.462894, -0.2140364, 2.0],
  [51.4630861, -0.1573428, 1.0],
  [51.4633719, -0.1819127, 2.0],
  [51.4638572, -0.1147675, 1.0],
  [51.4638742, -0.1678313, 5.0],
  [51.4644993, -0.1635286, 2.0],
  [51.4649342, -0.1728829, 1.0],
  [51.467133700000005, -0.2156778140353591, 1.0],
  [51.4676805, -0.1742392, 1.0],
  [51.4678064, -0.2099129, 1.0],
  [51.4687296, -0.1463346, 3.0],
  [51.469065400000005, -0.1942682667926211, 1.0],
  [51.4691875, -0.1132071, 1.0],

##### Visualize the Start trips of cycles interactively

In [37]:
from folium.plugins import HeatMapWithTime
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour_list, radius=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map